## Initialize the Env

In [1]:
import numpy as np

from datetime import datetime
import functools
from IPython.display import HTML
import jax
from jax import numpy as jp
import numpy as np
from typing import Any, Dict, Sequence, Tuple, Union, Optional
import wandb

from brax import base
from brax import envs
from brax import math
from brax.base import Base, Motion, Transform
from brax.envs.base import Env, MjxEnv, State
from brax.mjx.base import State as MjxState
from brax.training.agents.ppo import train as ppo
from brax.training.agents.ppo import networks as ppo_networks
from brax.io import html, mjcf, model

from etils import epath
from flax import struct
from matplotlib import pyplot as plt
import mediapy as media
from ml_collections import config_dict
import mujoco
from mujoco import mjx
import os

import yaml
from typing import List, Dict, Text

In [1]:
# ## TODO:
# 
# - Check the healthy z-range of the rodent. Now the training
#     - Check mj_data and how to pull out kinematics of the simulations
# - Check the `brax.envs` and how I can pass the custom parameters

# In[3]:


def load_params(param_path: Text) -> Dict:
    with open(param_path, "rb") as file:
        params = yaml.safe_load(file)
    return params


params = load_params("params/params.yaml")

class Rodent(MjxEnv):
    
    # Might want to change the terminate_when_unhealthy params to enables
    # longer episode length, since the average episode length is too short (1 timestep)
    # temp change the `terminate_when_unhealthy` to extend the episode length.
    def __init__(
            self,
            forward_reward_weight=5,
            ctrl_cost_weight=0.1,
            healthy_reward=0.5,
            terminate_when_unhealthy=False,
            healthy_z_range=(0.2, 1.0),
            reset_noise_scale=1e-2,
            exclude_current_positions_from_observation=False,
            **kwargs,
    ):
        params = load_params("params/params.yaml")
        mj_model = mujoco.MjModel.from_xml_path(params["XML_PATH"])
        mj_model.opt.solver = mujoco.mjtSolver.mjSOL_CG
        mj_model.opt.iterations = 6
        mj_model.opt.ls_iterations = 6

        physics_steps_per_control_step = 5
        kwargs['n_frames'] = kwargs.get(
            'n_frames', physics_steps_per_control_step)

        super().__init__(model=mj_model, **kwargs)

        self._forward_reward_weight = forward_reward_weight
        self._ctrl_cost_weight = ctrl_cost_weight
        self._healthy_reward = healthy_reward
        self._terminate_when_unhealthy = terminate_when_unhealthy
        self._healthy_z_range = healthy_z_range
        self._reset_noise_scale = reset_noise_scale
        self._exclude_current_positions_from_observation = (
            exclude_current_positions_from_observation
        )
        self.model=mj_model

    def reset(self, rng: jp.ndarray) -> State:
        """Resets the environment to an initial state."""
        rng, rng1, rng2 = jax.random.split(rng, 3)

        low, hi = -self._reset_noise_scale, self._reset_noise_scale
        qpos = self.sys.qpos0 + jax.random.uniform(
            rng1, (self.sys.nq,), minval=low, maxval=hi
        )
        qvel = jax.random.uniform(
            rng2, (self.sys.nv,), minval=low, maxval=hi
        )

        data = self.pipeline_init(qpos, qvel)

        obs = self._get_obs(data.data, jp.zeros(self.sys.nu))
        reward, done, zero = jp.zeros(3)
        metrics = {
            'forward_reward': zero,
            'reward_linvel': zero,
            'reward_quadctrl': zero,
            'reward_alive': zero,
            'x_position': zero,
            'y_position': zero,
            'distance_from_origin': zero,
            'x_velocity': zero,
            'y_velocity': zero,
        }
        return State(data, obs, reward, done, metrics)

    def step(self, state: State, action: jp.ndarray) -> State:
        """Runs one timestep of the environment's dynamics."""
        data0 = state.pipeline_state
        data = self.pipeline_step(data0, action)
        # based on the timestep simulation, calculate the rewards
        com_before = data0.data.subtree_com[1]
        com_after = data.data.subtree_com[1]
        velocity = (com_after - com_before) / self.dt
        forward_reward = self._forward_reward_weight * velocity[0]

        min_z, max_z = self._healthy_z_range
        is_healthy = jp.where(data.q[2] < min_z, 0.0, 1.0)
        is_healthy = jp.where(data.q[2] > max_z, 0.0, is_healthy)
        if self._terminate_when_unhealthy:
            healthy_reward = self._healthy_reward
        else:
            healthy_reward = self._healthy_reward * is_healthy

        ctrl_cost = self._ctrl_cost_weight * jp.sum(jp.square(action))

        obs = self._get_obs(data.data, action)
        reward = forward_reward + healthy_reward - ctrl_cost
        # terminates when unhealthy
        done = 1.0 - is_healthy if self._terminate_when_unhealthy else 0.0
        state.metrics.update(
            forward_reward=forward_reward,
            reward_linvel=forward_reward,
            reward_quadctrl=-ctrl_cost,
            reward_alive=healthy_reward,
            x_position=com_after[0],
            y_position=com_after[1],
            distance_from_origin=jp.linalg.norm(com_after),
            x_velocity=velocity[0],
            y_velocity=velocity[1],
        )

        return state.replace(
            pipeline_state=data, obs=obs, reward=reward, done=done
        )

    def _get_obs(
            self, data: mjx.Data, action: jp.ndarray
    ) -> jp.ndarray:
        """Observes humanoid body position, velocities, and angles."""
        position = data.qpos
        if self._exclude_current_positions_from_observation:
            position = position[2:]
            
        # external_contact_forces are excluded
        return jp.concatenate([
            position,
            data.qvel,
            data.cinert[1:].ravel(),
            data.cvel[1:].ravel(),
            data.qfrc_actuator,
        ])


# ## training loop

# In[5]:


envs.register_environment('rodent', Rodent)

# instantiate the environment
env_name = 'rodent'

## Simple Rodent Model

In [3]:
class Rodent(MjxEnv):
    
    # Might want to change the terminate_when_unhealthy params to enables
    # longer episode length, since the average episode length is too short (1 timestep)
    # temp change the `terminate_when_unhealthy` to extend the episode length.
    def __init__(
            self,
            forward_reward_weight=5,
            ctrl_cost_weight=0.1,
            healthy_reward=0.5,
            terminate_when_unhealthy=False,
            healthy_z_range=(0.2, 1.0),
            reset_noise_scale=1e-2,
            exclude_current_positions_from_observation=False,
            **kwargs,
    ):
        mj_model = mujoco.MjModel.from_xml_path("./models/simple_rodent.xml")
        mj_model.opt.solver = mujoco.mjtSolver.mjSOL_CG
        mj_model.opt.iterations = 6
        mj_model.opt.ls_iterations = 6

        physics_steps_per_control_step = 5
        kwargs['n_frames'] = kwargs.get(
            'n_frames', physics_steps_per_control_step)

        super().__init__(model=mj_model, **kwargs)

        self._forward_reward_weight = forward_reward_weight
        self._ctrl_cost_weight = ctrl_cost_weight
        self._healthy_reward = healthy_reward
        self._terminate_when_unhealthy = terminate_when_unhealthy
        self._healthy_z_range = healthy_z_range
        self._reset_noise_scale = reset_noise_scale
        self._exclude_current_positions_from_observation = (
            exclude_current_positions_from_observation
        )

    def reset(self, rng: jp.ndarray) -> State:
        """Resets the environment to an initial state."""
        rng, rng1, rng2 = jax.random.split(rng, 3)

        low, hi = -self._reset_noise_scale, self._reset_noise_scale
        qpos = self.sys.qpos0 + jax.random.uniform(
            rng1, (self.sys.nq,), minval=low, maxval=hi
        )
        qvel = jax.random.uniform(
            rng2, (self.sys.nv,), minval=low, maxval=hi
        )

        data = self.pipeline_init(qpos, qvel)

        obs = self._get_obs(data.data, jp.zeros(self.sys.nu))
        reward, done, zero = jp.zeros(3)
        metrics = {
            'forward_reward': zero,
            'reward_linvel': zero,
            'reward_quadctrl': zero,
            'reward_alive': zero,
            'x_position': zero,
            'y_position': zero,
            'distance_from_origin': zero,
            'x_velocity': zero,
            'y_velocity': zero,
        }
        return State(data, obs, reward, done, metrics)

    def step(self, state: State, action: jp.ndarray) -> State:
        """Runs one timestep of the environment's dynamics."""
        data0 = state.pipeline_state
        data = self.pipeline_step(data0, action)
        # based on the timestep simulation, calculate the rewards
        com_before = data0.data.subtree_com[1]
        com_after = data.data.subtree_com[1]
        velocity = (com_after - com_before) / self.dt
        forward_reward = self._forward_reward_weight * velocity[0]

        min_z, max_z = self._healthy_z_range
        is_healthy = jp.where(data.q[2] < min_z, 0.0, 1.0)
        is_healthy = jp.where(data.q[2] > max_z, 0.0, is_healthy)
        if self._terminate_when_unhealthy:
            healthy_reward = self._healthy_reward
        else:
            healthy_reward = self._healthy_reward * is_healthy

        ctrl_cost = self._ctrl_cost_weight * jp.sum(jp.square(action))

        obs = self._get_obs(data.data, action)
        reward = forward_reward + healthy_reward - ctrl_cost
        # terminates when unhealthy
        done = 1.0 - is_healthy if self._terminate_when_unhealthy else 0.0
        state.metrics.update(
            forward_reward=forward_reward,
            reward_linvel=forward_reward,
            reward_quadctrl=-ctrl_cost,
            reward_alive=healthy_reward,
            x_position=com_after[0],
            y_position=com_after[1],
            distance_from_origin=jp.linalg.norm(com_after),
            x_velocity=velocity[0],
            y_velocity=velocity[1],
        )
        return state.replace(
            pipeline_state=data, obs=obs, reward=reward, done=done
        )

    # this get observation might be faulty?
    def _get_obs(
            self, data: mjx.Data, action: jp.ndarray
    ) -> jp.ndarray:
        """Observes humanoid body position, velocities, and angles."""
        position = data.qpos
        if self._exclude_current_positions_from_observation:
            position = position[2:]
        
        # external_contact_forces are excluded
        return jp.concatenate([
            position,
            data.qvel,
            data.qfrc_actuator,
        ])


# ## training loop

# In[5]:


envs.register_environment('simple_rodent', Rodent)

# instantiate the environment
env_name = 'simple_rodent'
env = envs.get_environment(env_name)

## Load the Checkpoint

In [8]:
from brax.training.agents.ppo import networks as brax_networks

# Copy and pasted from https://github.com/google/brax/discussions/403#discussioncomment-7287194
def make_inference_fn(
    observation_size: int,
    action_size: int,
    normalize_observations: bool = False,
    network_factory_kwargs: Optional[Dict[str, Any]] = None,
):
    normalize = lambda x, y: x
    ppo_network = brax_networks.make_ppo_networks(
      observation_size,
      action_size,
      preprocess_observations_fn=normalize,
      **(network_factory_kwargs or {}),
    )
    make_policy = brax_networks.make_inference_fn(ppo_network)
    return make_policy

make_policy = make_inference_fn(
    observation_size=env.observation_size,
    action_size=env.action_size,
)


In [9]:
model_path = "./model_checkpoints/simple_rodent_573440"
params = model.load_params(model_path)
# jit_inference_fn = jax.jit(make_policy(params, deterministic=True))

In [6]:
# initialize a new policy with `make_policy`
params

(RunningStatisticsState(mean=Array([ 4.82222922e-02,  8.78764689e-03,  3.46816853e-02,  1.56694353e-01,
        -9.48220566e-02,  3.54142725e-01,  5.02438098e-03, -1.38106614e-01,
         1.50427055e-02,  1.31712770e-02,  9.42553133e-02,  3.23630683e-02,
         9.37415473e-03, -5.80575392e-02,  2.50213802e-01, -2.79280484e-01,
         1.14931084e-01,  6.14095449e-01,  3.83818150e-01, -1.17522448e-01,
         2.03906775e-01, -1.86026275e-01,  1.46754384e-01,  7.03811765e-01,
         4.03963715e-01, -8.50836933e-02, -3.71875172e-03, -1.05571583e-01,
        -2.69379467e-03, -1.34271130e-01,  2.30388995e-03, -1.17573068e-01,
         2.65469193e-03, -1.07484609e-01,  3.05168307e-03, -9.22036245e-02,
         2.25714245e-03, -7.72898495e-02,  6.29828265e-03, -1.06104948e-02,
         1.19147440e-02, -1.77078862e-02,  9.54281166e-03, -6.43372349e-03,
         5.65244723e-03,  6.63879793e-03,  2.58727884e-03,  5.66176418e-03,
         1.86694763e-03,  1.20356344e-02,  2.74863746e-03, -

## Render the Clip

In [13]:
from tqdm import tqdm

# define the jit reset/step functions
jit_reset = jax.jit(env.reset)
jit_step = jax.jit(env.step)
jit_inference_fn = jax.jit(make_policy(params, deterministic=True))

# initialize the state
rng = jax.random.PRNGKey(0)
state = jit_reset(rng)
rollout = [state.pipeline_state]

# grab a trajectory
n_steps = 250
render_every = 1

# might becasue brax does not clip the action to the xml limit in the model
for i in tqdm(range(n_steps)):
    act_rng, rng = jax.random.split(rng)
    ctrl, _ = jit_inference_fn(state.obs, act_rng)
    print(ctrl)
    state = jit_step(state, ctrl)
    rollout.append(state.pipeline_state)

    if state.done:
        break

media.show_video(env.render(rollout[::render_every], camera='side'), fps=1.0 / env.dt / render_every)

  0%|          | 0/250 [00:00<?, ?it/s]

[-0.05396793 -0.19769113 -0.01246248  0.05450987  0.12814136  0.00462889
 -0.01670834 -0.14408304 -0.03095406  0.02165836  0.17953786 -0.05386958]
{'distance_from_origin': Traced<ShapedArray(float32[])>with<DynamicJaxprTrace(level=1/0)>, 'forward_reward': Traced<ShapedArray(float32[])>with<DynamicJaxprTrace(level=1/0)>, 'reward_alive': Traced<ShapedArray(float32[], weak_type=True)>with<DynamicJaxprTrace(level=1/0)>, 'reward_linvel': Traced<ShapedArray(float32[])>with<DynamicJaxprTrace(level=1/0)>, 'reward_quadctrl': Traced<ShapedArray(float32[])>with<DynamicJaxprTrace(level=1/0)>, 'x_position': Traced<ShapedArray(float32[])>with<DynamicJaxprTrace(level=1/0)>, 'x_velocity': Traced<ShapedArray(float32[])>with<DynamicJaxprTrace(level=1/0)>, 'y_position': Traced<ShapedArray(float32[])>with<DynamicJaxprTrace(level=1/0)>, 'y_velocity': Traced<ShapedArray(float32[])>with<DynamicJaxprTrace(level=1/0)>}


  0%|          | 1/250 [01:06<4:37:34, 66.88s/it]

[ 0.27422085  0.99999994 -0.9999989  -0.99977624 -0.99999696 -0.99999994
 -0.99975854 -0.99999964  0.9983567   0.83791566  0.754382   -0.7818209 ]
{'distance_from_origin': Traced<ShapedArray(float32[])>with<DynamicJaxprTrace(level=1/0)>, 'forward_reward': Traced<ShapedArray(float32[])>with<DynamicJaxprTrace(level=1/0)>, 'reward_alive': Traced<ShapedArray(float32[], weak_type=True)>with<DynamicJaxprTrace(level=1/0)>, 'reward_linvel': Traced<ShapedArray(float32[])>with<DynamicJaxprTrace(level=1/0)>, 'reward_quadctrl': Traced<ShapedArray(float32[])>with<DynamicJaxprTrace(level=1/0)>, 'x_position': Traced<ShapedArray(float32[])>with<DynamicJaxprTrace(level=1/0)>, 'x_velocity': Traced<ShapedArray(float32[])>with<DynamicJaxprTrace(level=1/0)>, 'y_position': Traced<ShapedArray(float32[])>with<DynamicJaxprTrace(level=1/0)>, 'y_velocity': Traced<ShapedArray(float32[])>with<DynamicJaxprTrace(level=1/0)>}


  1%|          | 3/250 [02:13<2:29:31, 36.32s/it]

...

[ 0.9842653   0.99122864 -0.27537552 -0.8148661  -0.9820028  -0.67753595
 -0.9993369  -0.74805963  0.9873626   0.1076842   0.8574977  -0.18207113]
[ 0.9708802   0.8400163   0.47163495 -0.33388045 -0.65551347  0.05234361
 -0.9214596  -0.12971961  0.92430925 -0.51453197  0.67565435  0.3734113 ]


100%|█████████▉| 249/250 [02:39<00:00,  8.50it/s]

[ 0.54034585  0.04789154  0.59039587 -0.08017347 -0.09424862  0.17527027
 -0.07014687  0.3739842   0.66857237 -0.29773375  0.6259842   0.17408535]
[-0.25780192 -0.2941025   0.34329018 -0.27625692  0.13872913  0.09727278
  0.11956962  0.18283124  0.18283394 -0.0490406   0.32712165  0.02390905]


100%|██████████| 250/250 [02:40<00:00,  1.56it/s]
/opt/conda/lib/python3.10/site-packages/glfw/__init__.py:916: GLFWError: (65544) b'X11: The DISPLAY environment variable is missing'
  warnings.warn(message, GLFWError)
/opt/conda/lib/python3.10/site-packages/glfw/__init__.py:916: GLFWError: (65537) b'The GLFW library is not initialized'
  warnings.warn(message, GLFWError)


FatalError: an OpenGL platform library has not been loaded into this process, this most likely means that a valid OpenGL context has not been created before mjr_makeContext was called

In [17]:
import pickle
with open("rollout", "bw+") as f:
    pickle.dump(rollout, f)